**SPECIFIC FOR QUESTION ANSWERING TASK**

No need to run all the cell blocks, Directly run from the 40th Cell, for testing the model as per needs

In [ ]:
!pip install  datasets transformers
!pip uninstall accelerate -y
!pip uninstall transformers -y
!pip install -U accelerate
!pip install -U transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 21.7 MB/s eta 0:00:00


In [ ]:
#!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
import transformers

print(transformers.__version__)

4.36.2


In [ ]:
from datasets import load_dataset, load_metric
squad_v2 = False
model_checkpoint = "google/mobilebert-uncased"
batch_size = 16
datasets = load_dataset("squad_v2" if squad_v2 else "squad")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [ ]:
#function defined to view random elements in the dataset
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,id,title,context,question,answers
0,57300a06b2c2fd140056878d,"Charleston,_South_Carolina","The Charleston-North Charleston-Summerville Metropolitan Statistical Area consists of three counties: Charleston, Berkeley, and Dorchester. As of the 2013 U.S. Census, the metropolitan statistical area had a total population of 712,239 people. North Charleston is the second-largest city in the Charleston-North Charleston-Summerville Metropolitan Statistical Area and ranks as the third-largest city in the state; Mount Pleasant and Summerville are the next-largest cities. These cities combined with other incorporated and unincorporated areas along with the city of Charleston form the Charleston-North Charleston Urban Area with a population of 548,404 as of 2010. The metropolitan statistical area also includes a separate and much smaller urban area within Berkeley County, Moncks Corner (with a 2000 population of 9,123).",In which county is Moncks Corner located?,"{'text': ['Berkeley County'], 'answer_start': [763]}"
1,5724f8890a492a1900435691,Queen_Victoria,"At birth, Victoria was fifth in the line of succession after her father and his three older brothers: the Prince Regent, the Duke of York, and the Duke of Clarence (later William IV). The Prince Regent and the Duke of York were estranged from their wives, who were both past child-bearing age, so the two eldest brothers were unlikely to have any further children. The Dukes of Kent and Clarence married on the same day 12 months before Victoria's birth, but both of Clarence's daughters (born in 1819 and 1820 respectively) died as infants. Victoria's grandfather and father died in 1820, within a week of each other, and the Duke of York died in 1827. On the death of her uncle George IV in 1830, Victoria became heiress presumptive to her next surviving uncle, William IV. The Regency Act 1830 made special provision for the Duchess of Kent to act as regent in case William died while Victoria was still a minor. King William distrusted the Duchess's capacity to be regent, and in 1836 declared in her presence that he wanted to live until Victoria's 18th birthday, so that a regency could be avoided.","Whose infant daugheters both died, leaving him without an heir to the throne of England?","{'text': ['Duke of Clarence'], 'answer_start': [147]}"
2,56de6e7c4396321400ee28c0,Arnold_Schwarzenegger,"Gustav had a preference for his elder son, Meinhard (July 17, 1946 – May 20, 1971), over Arnold. His favoritism was ""strong and blatant,"" which stemmed from unfounded suspicion that Arnold was not his biological child. Schwarzenegger has said his father had ""no patience for listening or understanding your problems."" Schwarzenegger had a good relationship with his mother and kept in touch with her until her death. In later life, Schwarzenegger commissioned the Simon Wiesenthal Center to research his father's wartime record, which came up with no evidence of Gustav's being involved in atrocities, despite Gustav's membership in the Nazi Party and SA. Schwarzenegger's father's background received wide press attention during the 2003 California recall campaign. At school, Schwarzenegger was apparently in the middle but stood out for his ""cheerful, good-humored and exuberant"" character. Money was a problem in their household; Schwarzenegger recalled that one of the highlights of his youth was when the family bought a refrigerator.",What appliance did Schwarzenegger's family buy that he called a highlight of his youth?,"{'text': ['refrigerator'], 'answer_start': [1027]}"
3,57303f17b2c2fd1400568ae1,Roman_Republic,"The legal status of Roman citizenship was limited and was a vital prerequisite to possessing many important legal rights such as the right to trial and appeal, to marry, to vote, to hold office, to enter binding contracts, and to special tax exemptions. An adult male citizen with the full complement of legal and political rights was called ""optimo jure."" The optimo jure elected their assemblies, wh

Data Preprocessing

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
tokenizer("What is your name?", "My name is Sylvain.")

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102, 2026, 2171, 2003, 25353, 22144, 2378, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [ ]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:       #checking and isolating the first example
        break                                                                            #whose length > max_length
example = datasets["train"][i]

In [ ]:
len(tokenizer(example["question"], example["context"])["input_ids"])             #example length

396

In [ ]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])
#checking whether tokenizer is working as expected for the largest example

384

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [ ]:
[len(x) for x in tokenized_example["input_ids"]]         #multiple features created therefore returns multiple lengths

[384, 157]

In [ ]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] how many wins does the notre dame men's basketball team have? [SEP] the men's basketball team has over 1, 600 wins, one of only 12 schools who have reached that mark, and have appeared in 28 ncaa tournaments. former player austin carr holds the record for most points scored in a single game of the tournament with 61. although the team has never won the ncaa tournament, they were named by the helms athletic foundation as national champions twice. the team has orchestrated a number of upsets of number one ranked teams, the most notable of which was ending ucla's record 88 - game winning streak in 1974. the team has beaten an additional eight number - one teams, and those nine wins rank second, to ucla's 10, all - time in wins against the top team. the team plays in newly renovated purcell pavilion ( within the edmund p. joyce center ), which reopened for the beginning of the 2009 – 2010 season. the team is coached by mike brey, who, as of the 2014 – 15 season, his fifteenth at notr

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 3), (4, 8), (9, 13), (14, 18), (19, 22), (23, 28), (29, 33), (34, 37), (37, 38), (38, 39), (40, 50), (51, 55), (56, 60), (60, 61), (0, 0), (0, 3), (4, 7), (7, 8), (8, 9), (10, 20), (21, 25), (26, 29), (30, 34), (35, 36), (36, 37), (37, 40), (41, 45), (45, 46), (47, 50), (51, 53), (54, 58), (59, 61), (62, 69), (70, 73), (74, 78), (79, 86), (87, 91), (92, 96), (96, 97), (98, 101), (102, 106), (107, 115), (116, 118), (119, 121), (122, 126), (127, 138), (138, 139), (140, 146), (147, 153), (154, 160), (161, 165), (166, 171), (172, 175), (176, 182), (183, 186), (187, 191), (192, 198), (199, 205), (206, 208), (209, 210), (211, 217), (218, 222), (223, 225), (226, 229), (230, 240), (241, 245), (246, 248), (248, 249), (250, 258), (259, 262), (263, 267), (268, 271), (272, 277), (278, 281), (282, 285), (286, 290), (291, 301), (301, 302), (303, 307), (308, 312), (313, 318), (319, 321), (322, 325), (326, 330), (330, 331), (332, 340), (341, 351), (352, 354), (355, 363), (364, 373), (374,

In [ ]:
pad_on_right = tokenizer.padding_side == "right"  #for generalization

In [ ]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]         #needed for comparison as this will essentially act as the ground truth
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0                                                          #we are basically searching for the question in the context
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):           #can be expressed as  sequence_ids[token_start_index] = (0 if pad_on_right else 1)
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

Fine-tuning ( Training a pretrained model architecture for our custom dataset)

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/147M [00:00<?, ?B/s]

Some weights of MobileBertForQuestionAnswering were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)


In [ ]:
from transformers import default_data_collator
data_collator = default_data_collator

In [ ]:
import torch
from transformers import Trainer, TrainingArguments
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)


In [ ]:
#trainer.train()

In [ ]:
#trainer.save_model("test-squad-trained")

In [ ]:
from google.colab import drive           #skip
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#-----------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from transformers import AutoConfig, AutoModelForQuestionAnswering, AutoTokenizer

# Load the configuration
config_path = "/content/drive/MyDrive/test-squad-trained/config.json"
config = AutoConfig.from_pretrained(config_path)

# Load the tokenizer
model_checkpoint = "google/mobilebert-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Load the model from a local path
model_path = "/content/drive/MyDrive/test-squad-trained/model.safetensors"
model = AutoModelForQuestionAnswering.from_pretrained(model_path, config=config, local_files_only=True)


In [ ]:
'''import torch

for batch in trainer.get_eval_dataloader():
    break                                           #we only need a single batch for now
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}      #necessary because PyTorch models and tensors need to be on the same device
with torch.no_grad():                                #gradient calculation not needed as we just want a look at the output format
    output = trainer.model(**batch)                  #each k-v pair unpacked as a tensor
output.keys()'''

#The output of the model is a dict-like object that contains the loss (since we provided labels), the start and end logits. We won't need the loss for our predictions

odict_keys(['loss', 'start_logits', 'end_logits'])

In [ ]:
#output.start_logits.shape, output.end_logits.shape

(torch.Size([16, 384]), torch.Size([16, 384]))

In [ ]:
#testing on some random context and question
import torch

context = '''Not My Responsibility is a 2020 American short film written and produced by singer-songwriter Billie Eilish. A commentary on body shaming and double standards placed upon young women's appearances, it features a monologue from Eilish about the media scrutiny surrounding her body. The film is spoken-word and stars Eilish in a dark room, where she gradually undresses before submerging herself in black substance.
The film premiered during Eilish's Where Do We Go? World Tour on March 9, 2020, as a concert interlude, and was released online on May 26, 2020. Critics gave positive reviews, praising the commentary and tone, which they considered empowering.The second studio album, Happier Than Ever (2021). Some music journalists described it as the album's thematic centerpiece; others questioned its appearance on the tracklist, feeling that it lost its emotional impact without the visuals.'''
question = "Where did the film premier?"
inputs = tokenizer(question, context, return_tensors="pt")

# Forward pass to get the model's predictions
outputs = model(**inputs)
start_logits = outputs.start_logits  #getting the starting and ending logits(raw inputs)
end_logits = outputs.end_logits

# Use the tokenizer to decode the answer from the logits
answer_start = torch.argmax(start_logits)
answer_end = torch.argmax(end_logits) + 1
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

print("Question:", question)
print("Answer:", answer)



Question: Where did the film premier?
Answer: eilish ' s where do we go? world tour


In [ ]:
!pip install PyMuPDF


In [ ]:
#testing on PDF
import fitz

def extract_text_from_pdf(pdf_path):           #only extracting text
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(doc.page_count):
        page = doc[page_num]
        text += page.get_text()
    return text

def answer_question_from_pdf(question, pdf_path):
    # Extracting text from the PDF
    pdf_content = extract_text_from_pdf(pdf_path)

    # Tokenize input with a specified max length
    inputs = tokenizer(question, pdf_content, return_tensors="pt", max_length=512,truncation=True)

    # Forward pass to get the model's predictions
    outputs = model(**inputs)

    # Getting the start and end logits from the model's output
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    # Using argmax to get the predicted start and end positions
    start_position = start_logits.argmax(dim=-1).item()
    end_position = end_logits.argmax(dim=-1).item()

    # Extracting the answer span from the tokenized PDF content
    answer_span_tokens = inputs["input_ids"][0][start_position:end_position + 1]
    answer_span = tokenizer.decode(answer_span_tokens)

    return answer_span

question = "Which all programming languages do I know?"
pdf_path = "/content/drive/MyDrive/Rahul_Aryan_Resume.pdf"  # Replace with the path to your PDF file
# Get the answer
answer = answer_question_from_pdf(question, pdf_path)

print("Question:", question)
print("Answer:", answer)


Question: Which all programming languages do I know?
Answer: c / c + +, python, java, sql


In [ ]:
#print(output)

QuestionAnsweringModelOutput(loss=tensor(327160.6875), start_logits=tensor([[ 6.4303e+05,  3.7831e-01, -2.0990e+00,  ...,  4.8797e-01,
          7.4095e-01,  1.7906e+00],
        [ 6.4120e+05,  3.3578e-01, -2.0850e+00,  ...,  5.0028e-01,
          7.8234e-01,  1.6072e+00],
        [ 6.5340e+05, -9.8048e-01, -5.6608e-01,  ...,  1.1372e+00,
          1.4838e+00,  1.2724e+00],
        ...,
        [ 6.4316e+05, -1.0810e+00, -1.4882e+00,  ...,  6.3439e-01,
          1.6419e+00,  1.0901e+00],
        [ 6.4342e+05,  3.9496e-01, -1.1505e+00,  ...,  1.1186e+00,
          1.2069e+00,  8.3463e-01],
        [ 6.4593e+05, -1.3472e+00, -1.7759e+00,  ...,  6.3548e-01,
          1.0556e+00,  1.9148e+00]]), end_logits=tensor([[-3.0866e+06,  9.4177e-01, -8.5214e-01,  ...,  2.2420e-01,
         -1.7778e-02, -4.4957e-01],
        [-3.0493e+06,  9.2733e-01, -8.5908e-01,  ...,  9.7097e-03,
         -2.5956e-01, -3.0925e-01],
        [-3.5070e+06,  8.6003e-02, -4.5626e-01,  ..., -6.4281e-01,
         -7.632

#-----------------------------------------------------------------------------------------------------------------------------------------

Evaluation of the Model-

Since No evaluation metric has been used for now, the code blocks below are not of any use , but if needed evaluation metric can be added.

In [ ]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)
#works best for the 1st prediction , but if we need 2nd best or 3rd best prediction then we need a score parameter

(tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([ 18, 168, 165, 165,  21, 166, 165,  14, 166, 298, 166, 163, 167, 167,
         174, 168]))

In [ ]:
n_best_size = 20
import numpy as np

# Convert logits to NumPy arrays
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()

# Gather the indices of the best start/end logits
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()

# List to store valid answers
valid_answers = []

# Iterate through start and end indices
for start_index in start_indexes:
    for end_index in end_indexes:
        # Check if the start index is less than or equal to the end index
        if start_index <= end_index:
            # Create a dictionary for the valid answer
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": ""  # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )


In [ ]:
#modelled on a similar strategy as that of prepare_train_features()
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
raw_predictions = trainer.predict(validation_features)   #need to run this to finally get f1 score, but it will take more than 3 hrs

KeyboardInterrupt: ignored

In [ ]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))
#The Trainer hides the columns that are not used by the model (here example_id and offset_mapping which we will need for our post-processing), so we set them back

In [ ]:
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [ ]:
metric = load_metric("squad_v2" if squad_v2 else "squad")

<ipython-input-58-69495f532713>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad_v2" if squad_v2 else "squad")
